Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved. SPDX-License-Identifier: Apache-2.0

# Building a Social Network for Games on Amazon Neptune
This notebook shows how to use Amazon Neptune and Gremlin to build a graph of users and game titles for a games platform to make friends and games recommendations based on a user's network and games preferences.
## Background
Friends lists and similar ability to connect users remains a major ask I hear in games accounts. People who just want a simple plug in for AWS tooling. Lorem ipsum...

## Getting Started
In this section, we'll load the graph and set some visualization options. Then, we'll use Gremlin queries to inspect the data model used throughout the solution.

### Load data
The cell below loads the example social network into your Neptune cluster. When you run the cell, it will automatically install the `games-social-graph` dataset into your graph. This can take a few minutes.

In [1]:
%seed
# Source type: custom
# Language: gremlin
# Full File Query: False
# Location: Local
# Path: seed/
# Add this to Github repo --model Property_Graph --language gremlin --dataset games-social-graph --run

Dropdown(description='Source type:', options=('', 'samples', 'custom'), style=DescriptionStyle(description_wid…

Dropdown(description='Data model:', layout=Layout(display='none', visibility='hidden'), options=('', 'property…

Dropdown(description='Language:', layout=Layout(display='none', visibility='hidden'), options=('', 'opencypher…

Dropdown(description='Language:', layout=Layout(display='none', visibility='hidden'), options=('', 'opencypher…

Dropdown(description='Data set:', layout=Layout(display='none', visibility='hidden'), options=(), style=Descri…

...

Dropdown(description='Location:', layout=Layout(display='none', visibility='hidden'), options=('Local', 'S3'),…

FileChooser(path='/home/ec2-user/SageMaker', filename='', title='', show_hidden=False, select_desc='Select', c…

Button(description='Submit', layout=Layout(visibility='hidden'), style=ButtonStyle())

Output()

Output()

### Set visualization options
The cell below sets the property values to display on the graph visualization.

In [2]:
my_node_labels = '{"Game":"title","User":"username"}'
my_edge_labels = '{"rating":"stars"}'

The cell below configures the visualization to use specific colors and icons for the different parts of the data model.

In [3]:
%%capture
%%graph_notebook_vis_options

{
  "groups": {
    "Game": {
      "shape": "icon",
      "icon": {
        "face": "FontAwesome",
        "code": "\uf11b",
        "color": "#ff9900",
        "scaling": {
          "label": "true"
        }
      }
    },
    "User": {
      "shape": "icon",
      "icon": {
        "face": "FontAwesome",
        "code": "\uf2bd",
        "color": "#0749af",
        "scaling": {
          "label": "true"
        }
      }
    }
  }
}

### Data model
The dataset in our example is made up of 50 fictitious players and 50 fictitious game titles. A player is represented by a `User` vertex. Each `User` vertex has a property `username`. A game title is represented by the `Game` vertex. Each `Game` vertex has the properties `title`, `genre`, `ratingSum`, and `ratingCount`.

A `User` can have a `friendOf` edge to another `User`, representing that the first user is friends with the second user. A `User` can have a `rating` edge to a `Game` with a property `stars`, which represents a 1-to-5 star rating scale that the user gave to the game, where 5 is the most favorable rating.

**Disclaimer:** All game titles and usernames in this sample dataset were randomly generated and are fictional. Any resemblance to real-life entities, past or present, is purely coincidental.

### Visualize the data model
The cell below illustrates a graph with:
* 2 `User` vertices
* 2 `friendOf` edges connecting the 2 `User` vertices
* 1 `Game` vertex 
* 1 `rating` edge from a `User` vertex with property `star`

In [4]:
%%gremlin -d $my_node_labels -de $my_edge_labels

g.V('1050')
 .outE('friendOf')
 .inV().limit(1)
 .outE('friendOf')
 .inV().hasId('1050')
 .outE('rating').limit(1)
 .inV()
 .path().by(elementMap())

## Exploring the social network
Using the sample dataset, we can explore the relationships between players, friends, and game titles to make recommendations. For example, we can suggest new games for a player to try or new friends for a player to add.
### Suggest new friends with mutual friends
We can suggest new friends to a user by identifying people they are likely to know through their network. By traversing the graph, we look for second degree friends (friends of friends). We can rank the likelihood that a user might know a second degree friend by the number of mutual friends they share.

The cell below illustrates the graph with a `User`, their friends, and their second degree friends.

In [10]:
%%gremlin -d $my_node_labels -de $my_edge_labels

g.V('1034').as('user')
 .outE('friendOf')
 .inV().aggregate('friends') // get friends of user
 .outE('friendOf')
 .inV().where(P.neq('user')).where(P.without('friends')) // get friends of friends who are not user or friends of user
 .path().by(elementMap())

The cell below lists the top 3 second degree friends that a user is likely to know ranked by the number of mutual friends they share.

In [16]:
%%gremlin

g.V('1034').as('user')
 .outE('friendOf')
 .inV().aggregate('friends')
 .outE('friendOf')
 .inV().where(P.neq('user')).where(P.without('friends'))
 .groupCount().by('username')
 .order(local).by(values, desc)
 .unfold()
 .limit(3)

### Suggest new friends with similar preferences in games
We can suggest new friends to a user by identifying people that enjoyed playing the same games. By traversing the graph, we look for games that the user gave a 5-star rating, and then find other users who also gave the same games 5-star ratings. We can rank how likely they are to become friends by the number of games they mutually gave 5-star ratings.

The cell below illustrates the graph with a `User`, the games they gave 5-star ratings to, and other users who also gave those games 5-star ratings.

In [19]:
%%gremlin -d $my_node_labels -de $my_edge_labels

g.V('1011').as('user')
 .outE('rating').has('stars', 5)
 .inV() // games that user gave 5-star rating
 .inE('rating').has('stars', 5)
 .outV().where(neq('user')).not(inE('friendOf').outV().hasId('1011')) // other users that gave 5-star and is not user or friends of user
 .path().by(elementMap())

The cell below lists the top 3 users that share similar preference in games ranked by the number of games they both gave 5-star ratings.

In [18]:
%%gremlin

g.V('1011').as('user')
 .outE('rating').has('stars', 5)
 .inV()
 .inE('rating').has('stars', 5)
 .outV().where(neq('user')).not(inE('friendOf').outV().hasId('1011'))
 .groupCount().by('username')
 .order(local).by(values, desc)
 .unfold()
 .limit(3)

### Suggest new games rated highly by friends
We can suggest new games for a user to try by identifying games that the user's friends gave 5-star ratings. By traversing the graph, we look for a user's friends, and then find games that those friends gave 5-star ratings. We can rank the likelihood that the user might enjoy a game by the number of friends who gave 5-star ratings.

The cell below illustrates the graph with a `User`, their friends, and other games that received 5-star ratings from these friends.

In [20]:
%%gremlin -d $my_node_labels -de $my_edge_labels

g.V('1008').as('user')
 .outE('friendOf')
 .inV().as('friends') // friends of user
 .outE('rating').has('stars', 5)
 .inV().not(inE('rating').outV().hasId('1008')) // games that received 5-star ratings from friends but not user
 .path()
 .by(elementMap())

The cell below lists the top 3 games that the user is likely to enjoy ranked by the number of friends that gave 5-star ratings.

In [21]:
%%gremlin

g.V('1008').as('user')
 .outE('friendOf')
 .inV().as('friends')
 .outE('rating').has('stars', 5)
 .inV().not(inE('rating').outV().hasId('1008'))
 .groupCount().by('title')
 .order(local).by(values, desc)
 .unfold()
 .limit(3)

### Suggest new games rated highly by similar players
We can suggest new games for a user to try by identifying games that players with similar interests gave 5-star ratings. By traversing the graph, we look for a games that a user gave 5-star ratings, then find users that gave 5-star ratings to those games, and then look for other games that received 5-star ratings from those players. We can rank the likelihood that the user might enjoy a game by the number of similar players who gave 5-star ratings.

The cell below illustrates the graph with a `User`, their favorite games, other players who gave 5-star ratings to those games, and other games that received 5-star ratings from these players.

In [22]:
%%gremlin -d $my_node_labels -de $my_edge_labels

g.V('1026').as('user')
 .outE('rating').has('stars', 5)
 .inV() // games that user gave 5-star rating
 .inE('rating').has('stars', 5)
 .outV().where(neq('user')) // other users who gave the same games 5-star ratings
 .outE('rating').has('stars', 5)
 .inV().not(inE('rating').outV().hasId('1026')) // games that got 5-star ratings from other users but has not been rated by user
 .path()
 .by(elementMap())

The cell below lists the top 10 games that the user is likely to enjoy ranked by the number of similar players that gave 5-star ratings.

In [23]:
%%gremlin

g.V('1026').as('user')
 .outE('rating').has('stars', 5)
 .inV()
 .inE('rating').has('stars', 5)
 .outV().where(neq('user'))
 .outE('rating').has('stars', 5)
 .inV().not(inE('rating').outV().hasId('1026'))
 .groupCount().by('title')
 .order(local).by(values, desc)
 .unfold()
 .limit(10)

### Show friends who liked a game
When a user is considering a new game to play, we can show them a list of friends who gave the game a 5-star rating.

The cell below illustrates a graph with a `Game` that does not have a `rating` edge from a `User` and the friends of that user who have given a 5-star rating to the game.

In [24]:
%%gremlin -d $my_node_labels -de $my_edge_labels

g.V().hasLabel('Game').has('title','Blasterdroid')
 .inE('rating').has('stars', 5)
 .outV()
 .inE('friendOf')
 .outV().hasId('1002')
 .path()
 .by(elementMap())

The cell below lists the friends that gave a 5-star rating to a new game that a user is considering to play.

In [26]:
%%gremlin

g.V().hasLabel('Game').has('title','Blasterdroid')
 .inE('rating').has('stars', 5)
 .outV().filter(inE('friendOf').outV().hasId('1002'))
 .values('username')

## Conclusion
This notebook demonstrated how to use Amazon Neptune to build a social network for a games platform. We used a sample dataset with users and games and explored the friends relationship between users and the ratings connecting users and games. We traversed the graph to suggest new friends and games to users based on their existing friends and the games they rated.

### (Optional) Cleaning up
The cell below drops the example dataset from your graph.

In [103]:
%%gremlin

g.E().hasLabel('rating').drop().iterate()
g.E().hasLabel('friendOf').drop().iterate()
g.V().hasLabel('Game').drop().iterate()
g.V().hasLabel('User').drop().iterate()

## Additional Resources
The examples in this notebook showed how to develop a social network data model for a games platform. To build a social network solution using Amazon Neptune, we recommend the following resources:
* [Getting Started with Amazon Neptune](https://docs.aws.amazon.com/neptune/latest/userguide/graph-get-started.html)
* [Amazon Web Services Reference Architectures for Using Graph Databases](https://github.com/aws-samples/aws-dbs-refarch-graph/)
* [Amazon Neptune Developer Resources](https://aws.amazon.com/neptune/developer-resources/)